# Joining and Appending DataFrames in PySpark HW Solutions

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
import findspark

findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os

dfs = {}

base_path = r"Datasets/uw-madison-courses"

for f in os.listdir(base_path):
    f_name = f.split(".")[0]
    f_type = f.split(".")[-1]

    if f_type == "csv":
        dfs[f_name] = spark.read.csv(
            inferSchema=True,
            header=True,
            sep=",",
            path=f"{base_path}/{f}"
        )

Now check the contents of a few of the dataframes that were read in above.

In [3]:
dfs["grade_distributions"].limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
**Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [4]:
rooms = dfs["rooms"]
sections = dfs["sections"]

step1 = (
    rooms
    .join(
        sections,
        rooms.uuid == sections.room_uuid,
        how='left')
    .select(
        [rooms.uuid, rooms.facility_code, sections.course_offering_uuid, 'number']
    )
)

step1.limit(4).toPandas()

,uuid,facility_code,course_offering_uuid,number
0,ed828265-475b-31b4-b9a8-daec2a600449,0032,a71789f0-09d9-3374-bd21-04a719a62af8,3
1,ed828265-475b-31b4-b9a8-daec2a600449,0032,3c59e623-7e61-343a-951a-765cb62d3304,8
2,ed828265-475b-31b4-b9a8-daec2a600449,0032,dd3caaea-05bf-383a-afa4-52297c455208,7
3,ed828265-475b-31b4-b9a8-daec2a600449,0032,dd3caaea-05bf-383a-afa4-52297c455208,4


In [8]:
course_offerings = dfs["course_offerings"]

step2 = (
    step1
    .join(
        course_offerings,
        step1.course_offering_uuid == course_offerings.uuid,
        how='left')
    .select(
        ["name", rooms.uuid, rooms.facility_code, 'number', 'term_code']
    )
    .orderBy(
        "name"
    )
)

step2.toPandas()

,name,uuid,facility_code,number,term_code
0,"""""""Scand Modern"""" Phenomenon""",3215b86b-a4e4-39fe-9818-7e6e75569076,0400,1,1112
1,17th C Literature-1600-1660,b4bfb1f2-4ecd-361b-bb4f-a7f46e62602f,0048,1,1082
2,17th C Literature-1600-1660,165c677f-2362-3405-b219-c122ef48bfb9,0048,1,1072
3,17th-Century Literature,1047a29e-32b3-3a5b-8877-5cad2be098d1,0018,1,1152
4,18th Century British Novel,c25fbc34-2af7-3774-b306-34ebf6acfecf,0018,1,1112
...,...,...,...,...,...
209639,null,c00e22e2-ada4-3177-8094-a5f86dcdf714,0053,1,1092
209640,null,c00e22e2-ada4-3177-8094-a5f86dcdf714,0053,1,1164
209641,null,d3746b08-5fb6-3440-aa68-32dafbbe14ae,0153,1,1112
209642,null,d3746b08-5fb6-3440-aa68-32dafbbe14ae,0153,1,1144


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [10]:
step3 = step2.filter(step2.facility_code == "0469")
step3.toPandas()

,name,uuid,facility_code,number,term_code
0,18th-Century English Novel,9a15c09e-57d8-3658-ae72-aae5ffdef108,0469,1,1174
1,19th Century Philosophers,c731d9db-3bc4-3436-a5fd-7954098d3193,0469,1,1114
2,1st Yr Classical Chinese,f4ccf94a-5c59-3a6d-97e9-d44c4ca6a0fc,0469,1,1092
3,1st Yr Classical Chinese,f4ccf94a-5c59-3a6d-97e9-d44c4ca6a0fc,0469,1,1092
4,1st-Yr Seminar: Biological Sci,2b57b421-6c9f-3d88-9299-21b41c693446,0469,3,1152
...,...,...,...,...,...
16328,null,43a08e18-1ab4-397d-83c0-d0e9a0f2588e,0469,1,1094
16329,null,43a08e18-1ab4-397d-83c0-d0e9a0f2588e,0469,1,1092
16330,null,43a08e18-1ab4-397d-83c0-d0e9a0f2588e,0469,1,1074
16331,null,b80de0f7-d1e1-3084-8760-ab290d8f1011,0469,1,1172


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [11]:
subjects = dfs.get("subjects")
subject_memberships = dfs.get("subject_memberships")

step1 = (
    subjects
    .join(
        subject_memberships,
        subjects.code == subject_memberships.subject_code,
        how='inner')
    .select(
        ['name', 'course_offering_uuid']).withColumnRenamed('name', 'subject_name')
)

step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [12]:
step2 = (
    step1
    .join(
        sections,
        step1.course_offering_uuid == sections.course_offering_uuid,
        how='left'
    )
    .select(
        ['subject_name', 'room_uuid']
    )
)
step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
1,Materials Science and Engineering,3767a755-bf46-3d70-b607-1ded3f01fb18
2,Chemical and Biological Engineering,null
3,Electrical and Computer Engineering,null


In [18]:
# I added a filter to make this a little simpler
step3 = (
    step2
    .join(
        rooms,
        step2.room_uuid == rooms.uuid,
        how='left'
    )
    .select(
        ['subject_name', 'facility_code', 'room_code']
    )
)

step3.limit(4).toPandas()

AnalysisException: cannot resolve 'subject_code' given input columns: [facility_code, room_code, room_uuid, subject_name, uuid];
'Project [subject_name#455, facility_code#135, room_code#136, 'subject_code]
+- Join LeftOuter, (room_uuid#196 = uuid#134)
   :- Project [subject_name#455, room_uuid#196]
   :  +- Join LeftOuter, (course_offering_uuid#243 = course_offering_uuid#193)
   :     :- Project [name#221 AS subject_name#455, course_offering_uuid#243]
   :     :  +- Project [name#221, course_offering_uuid#243]
   :     :     +- Join Inner, (cast(code#220 as int) = subject_code#242)
   :     :        :- Relation [code#220,name#221,abbreviation#222] csv
   :     :        +- Relation [subject_code#242,course_offering_uuid#243] csv
   :     +- Relation [uuid#192,course_offering_uuid#193,section_type#194,number#195,room_uuid#196,schedule_uuid#197] csv
   +- Relation [uuid#134,facility_code#135,room_code#136] csv


In [19]:
# Option 1: Group by facility code and do a count
step3.groupBy('facility_code', 'subject_name').count().orderBy("facility_code", "subject_name").toPandas()  # False prevents truncation of column content

,facility_code,subject_name,count
0,None,ANIMAL HEALTH AND BIOMEDICAL SCIENCES,90
1,None,Accounting and Information Systems,187
2,None,Actuarial Science,2
3,None,African Languages and Literature,197
4,None,Afro-American Studies,145
...,...,...,...
4148,ONLINE,Rehabilitation Psychology and Special Education,29
4149,ONLINE,Religious Studies,2
4150,ONLINE,Social Work,59
4151,ONLINE,Sociology,4


In [24]:
# Option 2: Groupby subject name and pivot the facility code
# to see each facility side by side within each subject
step3.groupBy("subject_name").pivot("facility_code").count().show(10, False)

+---------------------------------+----+----+----+----+
|subject_name                     |0031|0140|0469|0545|
+---------------------------------+----+----+----+----+
|Asian American Studies           |null|6   |245 |24  |
|Religious Studies                |null|32  |282 |43  |
|HEBREW                           |null|7   |4   |null|
|Botany                           |null|4   |24  |2   |
|Urban and Regional Planning      |null|208 |17  |6   |
|Nutritional Sciences             |6   |null|null|28  |
|Kinesiology                      |1549|null|null|null|
|Art Education (Department of Art)|null|null|145 |null|
|Philosophy                       |1   |80  |1296|73  |
|Microbiology                     |null|null|1   |null|
+---------------------------------+----+----+----+----+
only showing top 10 rows



## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [20]:
grade_distributions = dfs["grade_distributions"]

grade_distributions.limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [22]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step1.toPandas()

,course_offering_uuid,sum(f_count)
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4
4,1ee15e7e-fa91-3205-8999-4cac58a95b61,0
...,...,...
80166,f4696bcb-a303-3b68-ac82-9b85ca228096,0
80167,cf3c9d45-cca6-341e-bdb1-31af018b781f,0
80168,e17aaf42-1411-3d9e-82a5-7e8610abcc88,0
80169,300e5a59-f23a-3d2b-9a03-47be0ff2edaa,0


In [24]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select(
    ['name', 'sum(f_count)']).orderBy("name", "sum(f_count)")
step2.toPandas()

,name,sum(f_count)
0,"""""""Scand Modern"""" Phenomenon""",0
1,17th C Literature-1600-1660,0
2,17th C Literature-1600-1660,0
3,17th-Century Literature,0
4,18th Century British Novel,0
...,...,...
80166,null,2
80167,null,2
80168,null,2
80169,null,2


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [8]:
values = [('Monkey', 10), ('Monkay', 36), ('Mnky', 123), \
          ('Elephant', 48), ('Elefant', 16), ('Ellafant', 1), \
          ('Hippopotamus', 48), ('Hipopotamus', 16), ('Hippo', 1)]
zoo = spark.createDataFrame(values, ['Animal', 'age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [14]:
# With the levenshtein distance!
from pyspark.sql.functions import *
from pyspark.sql.types import *

# First we create a dataframe with the 3 options we want to choose from
options = spark.createDataFrame(['Monkey', 'Elephant', 'Hippopotamus'], StringType())
options.show()

+------------+
|       value|
+------------+
|      Monkey|
|    Elephant|
|Hippopotamus|
+------------+



In [16]:
# And then we join the two dataframes together with a condition >5
results = zoo.join(options, levenshtein(zoo["Animal"], options["value"]) < 5, 'left')
results.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+



So we can see here that all of our values were correctly identified except for "Hippo" which was just way too different from "Hippopotamus" to get correctly identified. So this solution won't work for EVERY case, but we can see here that it did a great job correcting simple gramatical errors. 

### Great job! 